In [292]:
import sys
sys.path.append("..")
import imagenet_load
from importlib import reload
reload(imagenet_load)
import tarfile
import numpy as np
import imread
%pylab inline
import imread
from imread import imread_from_blob
from imagenet_load import orient
import multigpu
reload(multigpu)
import filter_gen
import conv
reload(conv)
import gc
import logging
import dill
from scipy.misc import imresize
import opt
reload(opt)
import exputil
reload(exputil)
from sklearn import metrics
import time
import scipy.linalg
from numba import jit
import concurrent.futures as fs

Populating the interactive namespace from numpy and matplotlib


/data/vaishaal/anaconda3/lib/python3.6/site-packages/IPython/core/magics/pylab.py:161: UserWarning: pylab import has clobbered these variables: ['imread']
`%matplotlib` prevents importing * from pylab and numpy
  "\n`%matplotlib` prevents importing * from pylab and numpy"


In [2]:
def preprocess(im):
    return imresize(im, size=(64,64,3))

In [3]:
%time loader = imagenet_load.ImagenetLoader(1000, classes_path="../classes", mmap_loc="/dev/shm/imagenet_train", n_procs=32, preprocess=preprocess, image_shape=(3, 64, 64))

%time loader_val = imagenet_load.ImagenetLoader(1000, classes_path="../classes", tarball_func=imagenet_load.VAL_LOADER, mmap_loc="/dev/shm/imagenet_val", n_procs=32, preprocess=preprocess, image_shape=(3, 64, 64))

There exists a matrix at /dev/shm/imagenet_train


CPU times: user 224 ms, sys: 1.94 s, total: 2.16 s
Wall time: 4.44 s


There exists a matrix at /dev/shm/imagenet_val


CPU times: user 176 ms, sys: 244 ms, total: 420 ms
Wall time: 591 ms


In [4]:
%time X_train = loader.load_all() / float(np.sqrt(64*64*3))
%time X_val = loader_val.load_all() / float(np.sqrt(64*64*3))

Loaded 0 classes into mem
Loaded 100 classes into mem
Loaded 200 classes into mem
Loaded 300 classes into mem
Loaded 400 classes into mem
Loaded 500 classes into mem
Loaded 600 classes into mem
Loaded 700 classes into mem
Loaded 800 classes into mem
Loaded 900 classes into mem
CPU times: user 59.9 s, sys: 31.7 s, total: 1min 31s
Wall time: 3min 33s
Loaded 0 classes into mem
Loaded 100 classes into mem
Loaded 200 classes into mem
Loaded 300 classes into mem
Loaded 400 classes into mem
Loaded 500 classes into mem
Loaded 600 classes into mem
Loaded 700 classes into mem
Loaded 800 classes into mem
Loaded 900 classes into mem
CPU times: user 2.7 s, sys: 4.09 s, total: 6.79 s
Wall time: 12 s


In [5]:
X_train = X_train.reshape(X_train.shape[0], -1).astype('float64') 
X_val = X_val.reshape(X_val.shape[0], -1).astype('float64') 

In [6]:
Y = np.eye(1000)[loader.Y.astype('int')]

In [7]:
%time XtX = X_train.T.dot(X_train)

CPU times: user 5h 31min 50s, sys: 1min 17s, total: 5h 33min 7s
Wall time: 5min 39s


In [8]:
%time Xty = X_train.T.dot(Y) 

CPU times: user 35min 19s, sys: 17min 47s, total: 53min 6s
Wall time: 50.6 s


In [14]:
opt.top_5_acc(loader.Y[:10, np.newaxis], Y[:10])

1.0

In [302]:
def compute_top_1_and_5_acc(labels, pred_weights):
    top1 = opt.top_k_accuracy(labels, pred_weights, k=1)
    top5 = opt.top_k_accuracy(labels, pred_weights, k=5)
    return top1, top5

def run_experiment_linear_64_64_reg_sweep(X_train, X_test, labels_train, labels_test, XtX, XtY, regs=[1e7, 1e8]): 
    solve_params_template = {"solver": "direct_linear"}
    feat_params = {
    'featurization_scheme': 'pixels_64x64_downsample_bilinear_interpolate',
    }
    diag_idx = np.diag_indices(XtX.shape[0])
    all_results = []
    for reg in regs:
        start = time.time()
        solve_params = solve_params_template.copy()
        solve_params["regularization"] = reg
        XtX[diag_idx] += reg
        try:
            model = scipy.linalg.solve(XtX, XtY, sym_pos=True)
        except:
            XtX[diag_idx] -= reg
            raise
        XtX[diag_idx] -= reg
        print("Done with solve..calculating metrics now")
        end = time.time()
        y_train_pred = X_train.dot(model)
        y_test_pred = X_test.dot(model)
        
        solve_time = end - start + 394
        top_1_train, top_5_train = compute_top_1_and_5_acc(labels_train, y_train_pred)
        top_1_test, top_5_test = compute_top_1_and_5_acc(labels_test, y_test_pred)
        results = {}
        results["train_top_1"] = top_1_train
        results["train_top_5"] = top_5_train
        results["test_top_1"] = top_1_test
        results["test_top_5"] = top_5_test
        results["solve_time"] = solve_time
        all_results.append((solve_params, feat_params.copy(), results, "simple run"))
    return all_results


    

In [ ]:
D = X_val.dot(X_train.T)
D *= -2
norms_sq_train = np.square(np.linalg.norm(X_train, axis=1))[:, np.newaxis]
norms_sq_test = np.square(np.linalg.norm(X_test, axis=1))[:, np.newaxis]
D += norms_sq_train.T
D += norms_sq_test




In [ ]:
def run_experiment_knn_64_64_k_sweep(X_train, X)

In [303]:
results = run_experiment_linear_64_64_reg_sweep(X_train, X_val, loader.Y, loader_val.Y, XtX, Xty)

Done with solve..calculating metrics now
Done with solve..calculating metrics now


In [304]:
for res in results:
    exputil.save_results(*res)

/bin/bash -c 'cd /data/vaishaal ; tar -cf ./picture_web_2017-04-04-02:31:30.281997.tar pictureweb --exclude-from <(find pictureweb -size +3M)'
775bd68fd3255abe1641b13c3bff6b83
/bin/bash -c 'cd /data/vaishaal ; tar -cf ./picture_web_2017-04-04-02:31:37.541449.tar pictureweb --exclude-from <(find pictureweb -size +3M)'
37a2da0d2467a0dcd8623d4b87b43638


In [ ]:
for r